In [ ]:
import gymnasium as gym
import miniworld
from matplotlib import pyplot as plt

miniworld_env_ids = [env_id for env_id in gym.registry if "MiniWorld" in env_id]

In [ ]:
print(miniworld_env_ids)

In [ ]:
def make_env():
    env = gym.make("MiniWorld-Maze-v0")
    return env

In [ ]:
"""
The observation space is an `ndarray` with shape `(obs_height, obs_width, 3)`
representing an RGB image of what the agents see.
"""

"""
class Actions(IntEnum):
      # Turn left or right by a small amount
      turn_left = 0
      turn_right = 1

      # Move forward or back by a small amount
      move_forward = 2
      move_back = 3

      # Pick up or drop an object being carried
      pickup = 4
      drop = 5

      # Toggle/activate an object
      toggle = 6

      # Done completing task
      done = 7
"""

env = make_env()

In [ ]:
print(env.action_space)
print(env.observation_space)
print(env.metadata)
obs, info = env.reset()
print(obs)
print(info)

print(env.get_wrapper_attr('min_x'))
print(env.get_wrapper_attr('max_x'))
print(env.get_wrapper_attr('min_z'))
print(env.get_wrapper_attr('max_z'))
plt.imshow(obs)

In [ ]:
# Turn left or right by a small amount
turn_left = 0
turn_right = 1

# Move forward or back by a small amount
move_forward = 2
move_back = 3

# Pick up or drop an object being carried
pickup = 4
drop = 5

# Toggle/activate an object
toggle = 6

# Done completing task
done = 7

obs, reward, terminated, truncated, info = env.step(1)
print(env.get_wrapper_attr('agent').pos)
print(env.get_wrapper_attr('agent').dir)
plt.imshow(obs)


In [ ]:
import gymnasium as gym
from vectorhash import build_vectorhash_architecture
from shifts import RatShift
from smoothing import RatSLAMSmoothing
import miniworld
from miniworld_agent import MiniworldVectorhashAgent
import math
import torch

### vhash
device = "cuda"
shapes = [(3, 3, 4), (4, 4, 5)]


def make_env():
    env = gym.make("MiniWorld-Maze-v0", max_episode_steps=-1)
    min_x = env.get_wrapper_attr("min_x")
    max_x = env.get_wrapper_attr("max_x")
    min_z = env.get_wrapper_attr("min_z")
    max_z = env.get_wrapper_attr("max_z")

    limits = torch.tensor([max_x - min_x, max_z - min_z, 2 * math.pi]).float()
    return env, limits


env, limits = make_env()
model = build_vectorhash_architecture(
    shapes,
    N_h=600,
    input_size=60 * 80,
    initalization_method="by_sparsity",
    limits=limits,
    device=device,
    shift=RatShift(),
    smoothing=RatSLAMSmoothing(device=device),
)


#### agent
agent = MiniworldVectorhashAgent(model, env)

In [ ]:
print(limits)

In [ ]:
path = [0] * 18 + [3] * 10

In [ ]:
import torch
from agent import path_test

results = path_test(agent, path, limits=torch.tensor([1, 1, 2 * math.pi / 8]))

In [ ]:
history = results[1]
ani = history.make_image_video()

In [ ]:
ani.save("gifs/testpath4.gif", progress_callback=lambda current, total: print(f"frame {current}/{total}"), dpi=150)